In [ ]:
from configs import models, env
from backend.llm.api import load_llm_langchain
from backend.prompts.templates import drug_interaction_prompt_template

config_loaded = {"model_config": models, "env": env}
llm = load_llm_langchain(source='ollama', model_name='LLaMA-3', config=config_loaded)

[LLM Loader] Successfully initialized model 'llama3' from 'ollama'.


In [2]:
def check_drug_interactions(drugs):
    # Normally this would hit DrugBank/OpenFDA
    risky_pairs = [
        ("aspirin", "warfarin"),
        ("lisinopril", "potassium"),
        ("atorvastatin", "clarithromycin"),
    ]
    interactions = []
    for i in range(len(drugs)):
        for j in range(i + 1, len(drugs)):
            pair = (drugs[i].lower(), drugs[j].lower())
            if pair in risky_pairs or pair[::-1] in risky_pairs:
                interactions.append(f"⚠️ {drugs[i]} + {drugs[j]}: Known severe interaction.")
    return interactions or ["✅ No known severe interactions found."]


In [3]:
medications = ["Aspirin", "Warfarin", "Metformin"]
interactions = check_drug_interactions(medications)

for msg in interactions:
    print(msg)

⚠️ Aspirin + Warfarin: Known severe interaction.


In [4]:
prompt = drug_interaction_prompt_template.format(
    drugs=", ".join(medications),
    interactions="\n".join(interactions)
)

In [5]:
response = llm.invoke(prompt)

print("📋 Safety Summary:\n")
print(response.content)

📋 Safety Summary:

**Safety Summary**

Combining Aspirin and Warfarin increases the risk of bleeding complications due to a synergistic effect on platelet function. This interaction can lead to severe hemorrhages, particularly in patients with pre-existing coagulopathy or those taking other anticoagulant medications.

**Suggested Monitoring Actions**

1. **Closely monitor INR (International Normalized Ratio)**: Regularly check INR levels every 2-3 days and adjust Warfarin dosage as needed to maintain a therapeutic range.
2. **Monitor platelet count**: Periodically check platelet counts to detect any potential thrombocytopenia.
3. **Assess bleeding risk**: Evaluate patients' bleeding risk factors, such as history of gastrointestinal bleeding or other coagulopathy-related issues.

**Warnings and Clinical Advice**

1. **Contraindicate combination in high-risk patients**: Avoid combining Aspirin and Warfarin in patients with a history of bleeding complications, gastrointestinal ulcers, or 